In [34]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import CTS as cts
from scipy.constants import *
from scipy.signal import hilbert

sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")
from data_analysis_utils import find_all_zerocrossing, hl_envelopes_idx

plt.rcParams.update({'font.size': 20})
%matplotlib qt

In [2]:
freq_in = [330e9,600e9] # Hz
cl = ['r', 'g', 'b']

ne = [1e12, 1e13, 5e13]
Te = [5,5, 300]
Ti = [1,1, 100]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel(r"$\omega_s - \omega_i (rad/s)$")
ax.set_ylabel(r"$S(k, \omega)$")

for i in range(len(ne)):

    lambda_in = c / freq_in[0]
    omega_in = 2*np.pi*freq_in[0]

    alpha, omega_arr, omega_in, Skw= cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=10000, ions="He+")
    
    ax.plot(omega_arr-omega_in, Skw, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}", color=cl[i])

    a =cts.compute_alpha(lambda_in*100, 180, ne[i], Te[i])
    print('S integral = ',integrate.simpson(Skw, omega_arr))
    print('alpha = ', a)

    lambda_in = c / freq_in[1]
    omega_in = 2*np.pi*freq_in[1]

    alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=1000, ions="He+")
    ax.plot(omega_arr-omega_in, Skw, '--', color=cl[i])


# Create first legend for the data lines
first_legend = ax.legend(loc='upper right', fontsize=16, bbox_to_anchor=(1, 1))
# Add the first legend manually to the axes
ax.add_artist(first_legend)
# Add a second legend for line styles
custom_lines = [Line2D([0], [0], color='black', lw=2),
                Line2D([0], [0], color='black', lw=2, linestyle='--')]
second_legend = ax.legend(custom_lines, [f'{freq_in[0]/1e9:.0f} GHz', f'{freq_in[1]/1e9:.0f} GHz'], 
                          loc='upper left', fontsize=20, bbox_to_anchor=(0, 1))
plt.tight_layout()

# fig, ax = plt.subplots(figsize=(16, 6))
# ax.set_xlabel(r"frequency (GHz)")
# ax.set_ylabel(r"$S(k, \omega)$")

# for i in range(len(ne)):
#     alpha, omega_arr, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="He+")
#     freq_arr = omega_arr/(2*np.pi) *1e-9
#     ax.plot(freq_arr, Skw, '--',label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
    
# ax.legend(loc='upper left')
# ax.set_title("Helium")

In [ ]:
lambda_in = 2*np.pi*c / omega_arr

plt.figure(figsize=(8, 6))
for i in range(len(ne)):
    a_ls =cts.compute_alpha(lambda_in*100, 180, ne[i], Te[i])
    plt.plot(omega_arr*1e-9, a_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")

plt.xlabel(r"frequency (GHz)")
plt.ylabel(r"$\alpha=1/k\lambda_D$")
plt.yticks(fontweight='bold')
plt.legend(loc='upper right', fontsize=16)
plt.tight_layout()

In [18]:
freq_ls = np.arange(300e9, 600e9, 10e9)
plt.figure(figsize=(8, 5))
plt.xlabel("frequency (GHz)")
plt.ylabel("$P_s/P_iL (1/cm) $")

for i in range(len(ne)):
    pr_ls = []
    for freq in freq_ls:

        lambda_in = c / freq
        alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="H+")
        pr = cts.power_ratio(ne[i], omega_arr, Skw, scattering_angle=180, L=1)
        pr_ls.append(pr)

    plt.plot(freq_ls*1e-9, pr_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
plt.legend()
plt.tight_layout()

In [2]:
# Parameters
f0 = 300e9 # units: Hz
n_cycles = 32
dt = 0.1 # units: ps
n_e = 1e13  # cm^-3
B = 0.1 # Tesla

x_0 = 0.5 # units: meter
x_pl = 0.1 # units: meter
L_arr = np.arange(x_0, x_0+x_pl, 0.01) # units: meter

In [3]:
plt.figure()
tarr, signal, envelope, x = cts.generate_thz_waveform(f0/1e12, 0.9, n_cycles, dt, 5, 100) # Units: THz, ps, #, ps, ps, #
print('wave covered distance = ', x*100, 'cm')
plt.plot(tarr, signal, linewidth=2, label='Input Signal')

tot_wave = np.zeros_like(tarr)
for L in L_arr:
    signal_propagated, fft_propagated, vgarr = cts.propagate_through_dispersive_medium(tarr, signal, L, n_e, debug=True)    
    tot_wave += signal_propagated / L**2

plt.plot(tarr, tot_wave, linewidth=2, label='Propagated Signal')

plt.xlabel('Time (ps)')
plt.ylabel('Amplitude')
plt.legend()
plt.legend()

wave covered distance =  0.8634022790399999 cm


In [24]:
zerocrossing_arr = find_all_zerocrossing(tot_wave, dt*1e-12, direction='positive')
omega_arr = 2*np.pi / np.diff(zerocrossing_arr)
new_Larr = np.linspace(0, x_pl, len(omega_arr))

In [25]:
ang_arr = cts.faraday_rotation_angle(omega_arr, n_e*1e6, B, new_Larr)

plt.figure(figsize=(8, 6))
plt.plot(new_Larr, ang_arr)
plt.title("Faraday rotation angle")
plt.xlabel("Distance (m)")
plt.ylabel("deg")
plt.tight_layout()

In [35]:
# Calculate envelope
envelope = np.abs(hilbert(tot_wave))

denv = np.diff(envelope)
max_idx = np.argmax(denv)

for i in range(max_idx, 0, -1):
    if denv[i] == 0 and denv[i-1] == 0:
        start_idx = i
        break

for i in range(max_idx, len(denv)):
    if denv[i] == 0 and denv[i+1] == 0:
        end_idx = i
        break

sig_amp = envelope[start_idx:end_idx]
L_interp = np.linspace(new_Larr[0], new_Larr[-1], len(sig_amp))
ang_interp = np.interp(L_interp, new_Larr, ang_arr)

In [36]:
plt.figure()
plt.plot(L_interp, ang_interp)
plt.plot(L_interp, sig_amp)
plt.plot(L_interp, ang_interp*sig_amp)